# Latent Dirichlet Allocation

In [19]:
import pandas as pd

data = pd.read_csv('data', sep=",", header=None)

data.columns = ['text']

data.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


The data is a collection of emails that are not labelled. Let's try extract topics from them!

## Preprocessing 

👇 You're used to it by now... Clean up! Store the cleaned text in a new dataframe column "clean_text".

In [51]:
from string import punctuation
import re
from nltk.corpus import stopwords

def clean_text(text):

    text = re.sub(r'http\S+', ' ', text)
    text = re.sub("\d+", " ", text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans("", "", punctuation))
    text = text.lower()
    
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in text if not w.lower() in stop_words]
    filtered_sentence = []
    for w in text:
        if w not in stop_words:
            filtered_sentence.append(w)
    return text

data['clean_text'] = data['text'].apply(clean_text)
data.head()

<>:8: DeprecationWarning: invalid escape sequence \d
<>:8: DeprecationWarning: invalid escape sequence \d
C:\Users\utilisateur\AppData\Local\Temp\ipykernel_14372\1067284699.py:8: DeprecationWarning: invalid escape sequence \d
  text = re.sub("\d+", " ", text)


,text,clean_text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,from gldcunixbcccolumbiaedu gary l dare subjec...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...,from atterlepvelaacsoaklandedu cardinal ximene...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...,from minerkuhubccukansedu subject re ancient b...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...,from atterlepvelaacsoaklandedu cardinal ximene...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...,from vzhivovsuperiorcarletonca vladimir zhivov...


## Latent Dirichlet Allocation model

👇 Train an LDA model to extract potential topics.

In [52]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel

dictionary = corpora.Dictionary(data['clean_text'].str.split().tolist())

doc_term_matrix = [dictionary.doc2bow(text) for text in data['clean_text'].str.split().tolist()]

# creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# runnug and trainig LDA model on the document term matrix for 3 topics
ldamodel = Lda(doc_term_matrix, num_topics = 3, id2word = dictionary, passes = 50)

dictionary

## Visualize potential topics

👇 The function to print the words associated with the potential topics is already made for you. You just have to pass the correct arguments!

In [53]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\utilisateur\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.159598 -0.036795       1        1  53.901784
0     -0.050300  0.052252       2        1  34.075725
2      0.209898 -0.015456       3        1  12.022491, topic_info=       Term          Freq         Total Category  logprob  loglift
152     the  18345.000000  18345.000000  Default  30.0000  30.0000
236     god   1195.000000   1195.000000  Default  29.0000  29.0000
149    team    608.000000    608.000000  Default  28.0000  28.0000
10       at   1281.000000   1281.000000  Default  27.0000  27.0000
151    that   5518.000000   5518.000000  Default  26.0000  26.0000
...     ...           ...           ...      ...      ...      ...
167    will    148.152668   1296.763622   Topic3  -5.7041  -0.0510
1195    nhl    121.772392    363.042384   Topic3  -5.9002   1.0260
653   first    121.974969    427.094113   Topic3  -5.8985   0.8652
164     was    134.426528   2211.849734   Topic3  -5.8013  -0.6822
171    with    113.719125   1850.930041   Topic3  -5.9686  -0.6713

[268 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.543623           a
0         2  0.384065           a
0         3  0.072285           a
2810      3  0.976676  adirondack
2661      3  0.995853         ahl
...     ...       ...         ...
881       2  0.687361        year
881       3  0.253589        year
327       1  0.670609         you
327       2  0.328568         you
327       3  0.001123         you

[389 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

## Predict topic of new text

👇 You can now use your LDA model to predict the topic of a new text. First, use your vectorizer to vectorize the example. Then, use your LDA model to predict the topic of the vectorized example.